# One-vs-All Classification

## Table of Contents
- [Introduction](#intro)
- [The dataset](#dataset)

<a id='intro'></a>
### Introduction
In a [previous article](https://github.com/marty-vanhoof/Maching_Learning/blob/master/Logistic_Regression/Logistic_Regression.ipynb), we talked in detail about logistic regression in the case of binary classification; that is, given a set of training data $\{ \, ( \mathbf{x}_1,y_1 ), ( \mathbf{x}_2,y_2 ) \ldots,  ( \mathbf{x}_m,y_m ) \, \}$, logistic regression finds the parameters $\theta_0, \theta_1, \ldots, \theta_n$ of a function $h_\theta(\mathbf{x})$ that estimates the probability of a class label $y=1$ given a feature vector $\mathbf{x}$, and so the function $1 - h_\theta(\mathbf{x})$ estimates the probability of a class label $y=0$ given $\mathbf{x}$.  We can then predict binary outcomes by specifying a probability threshold, and the threshold we use is given as follows

$$ \mathrm{prediction} =
\begin{cases}
1 & \mathrm{if} \quad P(y = 1 \,|\, \mathbf{x} ; \theta) \geq 0.5 \\ 
0 & \mathrm{if} \quad P(y = 0 \,|\, \mathbf{x} ; \theta) < 0.5
\end{cases} 
$$

One-vs-all classification extends logistic regression to the case of classifying more than two class labels by considering a sequence of binary classifications.  Suppose the target variable $y$ can be labeled in $k$ possible ways, so $y \in \{1, 2, \ldots, k \}$ with $k \geq 2$.  We can turn this into a sequence of binary classification problems and use logistic regression at each step.  

First separate the labels $\{1, 2, \dots, k \}$ into either 1 or $\{2, \dots, k \}$ and fit a standard logistic regression model to this.  This logistic regression model will find a function $h_\theta^{(1)}(\mathbf{x})$ that classifies the data into either 1 or $\{ 2, \ldots, k \}$.

Next, separate the labels into either 2 or $\{ 1, 3, \dots, k \}$ and fit another logistic regression model to this, resulting in a function $h_\theta^{(2)}(\mathbf{x})$ that classifies the data into either 2 or $\{ 1, 3, \dots, k \}$.
And so on...the last step will fit another logistic regression model that finds a function $h_\theta^{(k)}(\mathbf{x})$ that classifies the data into either $k$ or $\{ 1, 2, \ldots, k-1 \}$.

The result is that we have $k$ classifiers $h_\theta^{(1)}(\mathbf{x}), h_\theta^{(2)}(\mathbf{x}), \ldots, h_\theta^{(k)}(\mathbf{x})$ each trying to estimate the probability of a class label $y \in \{ 1, 2, \ldots, k \}$ given $\mathbf{x}$.  That is,

$$
h_\theta^{(i)}(\mathbf{x}) = P(y = i \,|\, \mathbf{x}; \theta) \,, \quad i = 1,2, \ldots, k
$$

Then to make a prediction on a feature vector $\mathbf{x}$, pick the class $i$ that gives maximum probability

$$
\max \limits_i h_\theta^{(i)}(\mathbf{x}) 
$$

A nice discussion of one-vs-all classification can be found in [this video](https://www.youtube.com/watch?v=BzSsQWhDRXE&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW).

<a id='dataset'></a>
### The Dataset
As usual, we will be working through an example from one of the programming exercises in Andrew Ng's machine learning course.  We will implement one-vs-all logistic regression to recognize hand-written digits.  The [dataset](https://github.com/marty-vanhoof/Maching_Learning/blob/master/data/logReg_data3.mat) contains 5000 training examples of handwritten digits, which is a subset of the MNIST database of handwritten digits available [here](http://yann.lecun.com/exdb/mnist/).  Each training example is a 20 pixel by 20 pixel grayscale image of the digit.  The 20 by 20 pixel grid can be represented as a 400 dimensional vector of floating-point numbers; each number representing the grayscale intensity of a pixel at that location.

In [2]:
import numpy as np  
import pandas as pd 
import os
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

filepath = os.getcwd() + '/logReg_data3.mat'
data = loadmat('logReg_data3.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [5]:
print(data['X'].shape, data['y'].shape)

(5000, 400) (5000, 1)
